## Random

### Configuración Inicial

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 1.26.0
Uninstalling numpy-1.26.0:
  Successfully uninstalled numpy-1.26.0
  Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have nump

In [2]:
!pip install scikit-surprise --no-build-isolation --no-deps
!pip install memory_profiler

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2708540 sha256=717d0581bacbe93e57ca9481a1cac11257a938017e6fc64eedb27c2d6ce27aec
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


### Instalación de Librerías

In [3]:
import time
import json
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
from memory_profiler import memory_usage
import itertools
import scipy.sparse as sparse
import random
import gdown
from surprise import SVDpp, Dataset, Reader, accuracy
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader



### Importación de los Datos

In [4]:
gdown.download(id='1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx', output='training_ratings.csv', quiet=False)
gdown.download(id='1oHo9HLB6SzeqZs76FCkfQ1irSQepqp16', output='validation_ratings.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx
From (redirected): https://drive.google.com/uc?id=1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx&confirm=t&uuid=43e68523-05df-4363-83e5-460731b989cc
To: /content/training_ratings.csv
100%|██████████| 205M/205M [00:01<00:00, 187MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oHo9HLB6SzeqZs76FCkfQ1irSQepqp16
To: /content/validation_ratings.csv
100%|██████████| 64.4M/64.4M [00:00<00:00, 69.9MB/s]


'validation_ratings.csv'

In [5]:
df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

In [6]:
# dataset mechanics
gdown.download(id='1cVGSLNVqxrAoKzeqxt_FfQ4Ggs9VvCDO', output='mechanics.csv', quiet=False)
df_mechanics = pd.read_csv('mechanics.csv')

Downloading...
From: https://drive.google.com/uc?id=1cVGSLNVqxrAoKzeqxt_FfQ4Ggs9VvCDO
To: /content/mechanics.csv
100%|██████████| 7.05M/7.05M [00:00<00:00, 79.8MB/s]


### Preprocesamiento de Datos

In [7]:
df_mechanics = pd.read_csv('mechanics.csv')
# Usamos BGGId como índice para que la búsqueda sea rápida
df_mechanics.set_index('BGGId', inplace=True)
print("Datos de mecánicas cargados y listos.")

# --- Calcular la popularidad de los ítems ---
# Usamos el dataframe de entrenamiento COMPLETO (df_train) para obtener una
# medida de popularidad global y precisa.
item_popularity = df_train['item'].value_counts().to_dict()
total_interactions = len(df_train)

# Convertimos las cuentas en probabilidades para el cálculo de novedad
item_popularity_prob = {item_id: count / total_interactions for item_id, count in item_popularity.items()}
print(f"Popularidad calculada para {len(item_popularity)} ítems.")

Datos de mecánicas cargados y listos.
Popularidad calculada para 16748 ítems.


In [8]:
def novelty_at_k(group, k, popularity_prob):
    """Calcula la Novedad@K para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    topk_items = group.head(k)['itemID']

    novelty_scores = []
    for item_id in topk_items:
        # Si un ítem no está en el diccionario de popularidad, se le asigna una probabilidad muy baja
        prob = popularity_prob.get(item_id, 1e-6)
        novelty_scores.append(-np.log2(prob))

    return np.mean(novelty_scores) if novelty_scores else 0.0

def diversity_at_k(group, k, mechanics_df):
    """Calcula la Diversidad@K (Intra-List Diversity) para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    topk_items = group.head(k)['itemID'].tolist()

    # Nos aseguramos de que los ítems recomendados tengan datos de mecánicas
    topk_items = [item for item in topk_items if item in mechanics_df.index]

    if len(topk_items) < 2:
        return 0.0

    item_vectors = mechanics_df.loc[topk_items].values

    # Calculamos la disimilitud del coseno (1 - similitud) para todos los pares de ítems
    dissimilarity_sum = 0
    num_pairs = 0
    for i in range(len(item_vectors)):
        for j in range(i + 1, len(item_vectors)):
            sim = cosine_similarity([item_vectors[i]], [item_vectors[j]])[0][0]
            dissimilarity_sum += (1 - sim)
            num_pairs += 1

    return dissimilarity_sum / num_pairs if num_pairs > 0 else 0.0



In [9]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [10]:
print(f"Tamaño original del training set: {len(df_train)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_train_sample = df_train.sample(n=1000000, random_state=42)
print(f"Tamaño del nuevo training set (muestra): {len(df_train_sample)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_val_sample = df_val.sample(n=50000, random_state=42)
print(f"Tamaño del nuevo validation set (muestra): {len(df_val_sample)}")

Tamaño original del training set: 10200445
Tamaño del nuevo training set (muestra): 1000000
Tamaño del nuevo validation set (muestra): 50000


In [11]:
def evaluar_random_topn(df_train, df_val, n=10, sample_per_user=50):
    """
    Genera recomendaciones aleatorias para cada usuario, tomando un sample
    limitado de items no vistos para evitar usar toda la matriz.
    """
    # Diccionario {usuario: items que ha visto}
    user2seen = df_train.groupby('user')['item'].apply(set).to_dict()

    # Lista de todos los items
    all_items = df_train['item'].unique().tolist()

    top_n = {}
    for uid in df_val['user'].unique():
        seen = user2seen.get(uid, set())
        # Items posibles para recomendar
        candidates = list(set(all_items) - seen)
        # Tomar un sample limitado
        sample_candidates = random.sample(candidates, min(sample_per_user, len(candidates)))
        # Tomar n recomendaciones aleatorias
        recs = random.sample(sample_candidates, min(n, len(sample_candidates)))
        top_n[uid] = [(iid, random.randint(1,5)) for iid in recs]

    return top_n


# Medir tiempo de ejecución
start_time = time.time()
top_n = evaluar_random_topn(df_train_sample, df_val_sample, n=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")

# Medir la memoria utilizada
memoria = memory_usage(
    (evaluar_random_topn, (df_val_sample, df_val_sample), {'n':10})
)
print("Memoria usada (MB):", max(memoria) - min(memoria))

Tiempo de ejecución: 51.87 segundos
Memoria usada (MB): 36.90625


In [12]:
def rmse_mae_from_topn(top_n, df_val_sample):
    real, predicho = [], []
    total = sum(len(recs) for recs in top_n.values())
    i = 0
    for uid, recs in top_n.items():
      for iid, pred in recs:
        real_vals = df_val_sample.loc[(df_val_sample['user'] == uid) & (df_val_sample['item'] == iid), 'rating']
        if not real_vals.empty:
            real.append(real_vals.values[0])
            predicho.append(pred)
        i += 1
        if i % 10000 == 0 or i == total:  # muestra cada 100 pasos o al final
            progreso = (i / total) * 100
            print(f"Progreso: {i}/{total} ({progreso:.2f}%)")

    return math.sqrt(mean_squared_error(real, predicho)), mean_absolute_error(real, predicho)

rmse, mae = rmse_mae_from_topn(top_n, df_val_sample)
print("RMSE para las top n recomendaciones", rmse)
print("MAE para las top n recomendaciones", mae)


Progreso: 10000/375480 (2.66%)
Progreso: 20000/375480 (5.33%)
Progreso: 30000/375480 (7.99%)
Progreso: 40000/375480 (10.65%)
Progreso: 50000/375480 (13.32%)
Progreso: 60000/375480 (15.98%)
Progreso: 70000/375480 (18.64%)
Progreso: 80000/375480 (21.31%)
Progreso: 90000/375480 (23.97%)
Progreso: 100000/375480 (26.63%)
Progreso: 110000/375480 (29.30%)
Progreso: 120000/375480 (31.96%)
Progreso: 130000/375480 (34.62%)
Progreso: 140000/375480 (37.29%)
Progreso: 150000/375480 (39.95%)
Progreso: 160000/375480 (42.61%)
Progreso: 170000/375480 (45.28%)
Progreso: 180000/375480 (47.94%)
Progreso: 190000/375480 (50.60%)
Progreso: 200000/375480 (53.27%)
Progreso: 210000/375480 (55.93%)
Progreso: 220000/375480 (58.59%)
Progreso: 230000/375480 (61.25%)
Progreso: 240000/375480 (63.92%)
Progreso: 250000/375480 (66.58%)
Progreso: 260000/375480 (69.24%)
Progreso: 270000/375480 (71.91%)
Progreso: 280000/375480 (74.57%)
Progreso: 290000/375480 (77.23%)
Progreso: 300000/375480 (79.90%)
Progreso: 310000/37548

In [13]:
from sklearn.metrics import ndcg_score

# --- Crear DataFrame de Evaluación con Scores Aleatorios ---
print("Creando DataFrame de evaluación para el modelo 'Random'...")
# Usamos la muestra del set de validación, como en tu notebook original
df_eval = df_val_sample.copy()
df_eval = df_eval.rename(columns={'user': 'userID', 'item': 'itemID'})

# Creamos el 'label' para saber si el ítem es relevante
df_eval['label'] = (df_eval['rating'] >= 7).astype(int)
# Asignamos un score completamente aleatorio a cada interacción
np.random.seed(42)
df_eval['score'] = np.random.rand(len(df_eval))
print("DataFrame de evaluación aleatorio creado con éxito.")

# --- Funciones de métrica de ranking (puedes moverlas si ya las tienes en otra celda) ---
def precision_recall_at_k(group, k):
    group = group.sort_values('score', ascending=False)
    topk = group.head(k)
    hits = topk['label'].sum()
    total_relevant = group['label'].sum()
    precision = hits / k if k > 0 else 0
    recall = hits / total_relevant if total_relevant > 0 else 0
    return precision, recall

def ndcg_at_k(group, k):
    if group['label'].sum() == 0: return 0.0
    ranked_group = group.sort_values('score', ascending=False).head(k)
    if len(ranked_group) < 2: return 0.0
    true_relevance = np.asarray([ranked_group['label'].values])
    predicted_scores = np.asarray([ranked_group['score'].values])
    return ndcg_score(true_relevance, predicted_scores)


# --- Evaluación Individual Completa ---
K_values = [10]
individual_results = []
print("\nCalculando métricas individuales para Random...")

grouped_users = df_eval.groupby('userID')

for k in K_values:
    metrics = grouped_users.apply(lambda x: precision_recall_at_k(x, k))
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])
    avg_ndcg = grouped_users.apply(lambda x: ndcg_at_k(x, k)).mean()
    avg_novelty = grouped_users.apply(lambda x: novelty_at_k(x, k, item_popularity_prob)).mean()
    avg_diversity = grouped_users.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()

    individual_results.append({
        'K': k,
        'Precision@K': avg_precision,
        'Recall@K': avg_recall,
        'nDCG@K': avg_ndcg,
        'Novelty@K': avg_novelty,
        'Diversity@K': avg_diversity
    })

individual_results_df = pd.DataFrame(individual_results)
print("\n--- Resultados de Evaluación Individual (Random) ---")
print(individual_results_df)

Creando DataFrame de evaluación para el modelo 'Random'...
DataFrame de evaluación aleatorio creado con éxito.

Calculando métricas individuales para Random...


/tmp/ipython-input-3113504245.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_users.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3113504245.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_ndcg = grouped_users.apply(lambda x: ndcg_at_k(x, k)).mean()
/tmp/ipython-input-3113504245.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping colum


--- Resultados de Evaluación Individual (Random) ---
    K  Precision@K  Recall@K    nDCG@K  Novelty@K  Diversity@K
0  10     0.089331  0.745785  0.166149  11.324084     0.187942


/tmp/ipython-input-3113504245.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_diversity = grouped_users.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()


Ahora haremos lo mismo pero para un grupo de 4 personas que han calificado ese item (haremos 4 personas por mas que haya juegos que es de máximo 2 o 10 etc por simplicidad). Lo que hacemos es calcular el promedio real de las calificaciones de esas 4 personas y compararlo contra una predicción grupal generada de manera completamente aleatoria en el rango de ratings posibles.

In [14]:
def evaluar_random_topn_grupos(df_train, df_val, n=10, sample_per_user=50):
    """
    Genera recomendaciones aleatorias para cada usuario, tomando un sample
    limitado de items no vistos para evitar usar toda la matriz.
    """
    # Diccionario {usuario: items que ha visto}
    user2seen = df_train.groupby('user')['item'].apply(set).to_dict()

    # Lista de todos los items
    all_items = df_train['item'].unique().tolist()

    top_n = {}
    # armar grupos
    usuarios = df_val['user'].unique()
    grupos = [usuarios[i:i+4] for i in range(0, len(usuarios) - len(usuarios)%4, 4)]

    seen_group = set()
    for grupo in grupos:
      for u in grupo:
          seen_group |= user2seen.get(u, set())
          # Items posibles para recomendar
          candidates = list(set(all_items) - seen_group)
          # Tomar un sample limitado
          sample_candidates = random.sample(candidates, min(sample_per_user, len(candidates)))
          # Tomar n recomendaciones aleatorias
          recs = random.sample(sample_candidates, min(n, len(sample_candidates)))
          top_n[tuple(grupo)] = [(iid, random.randint(1,5)) for iid in recs]

    return top_n


# Medir tiempo de ejecución
start_time = time.time()
top_n_grupo = evaluar_random_topn_grupos(df_train_sample, df_val_sample, n=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")

# Medir la memoria utilizada
memoria = memory_usage(
    (evaluar_random_topn_grupos, (df_train_sample, df_val_sample), {'n':10})
)
print("Memoria usada (MB):", max(memoria) - min(memoria))

Tiempo de ejecución: 63.00 segundos
Memoria usada (MB): 108.1875


In [15]:
def rmse_mae_from_topn_grupo(top_n_grupo, df_val_sample):
    real, predicho = [], []
    total = sum(len(recs) for recs in top_n.values())
    i = 0
    for grupo, recs in top_n_grupo.items():
      for iid, pred in recs:
        real_vals = df_val_sample.loc[(df_val_sample['user'].isin(grupo)) & (df_val_sample['item'] == iid), 'rating']
        if not real_vals.empty:
            real.append(real_vals.values[0])
            predicho.append(pred)
        i += 1
        if i % 10000 == 0 or i == total:  # muestra cada 100 pasos o al final
            progreso = (i / total) * 100
            print(f"Progreso: {i}/{total} ({progreso:.2f}%)")

    return math.sqrt(mean_squared_error(real, predicho)), mean_absolute_error(real, predicho)

rmse_grupo, mae_grupo = rmse_mae_from_topn_grupo(top_n_grupo, df_val_sample)
print("RMSE para las top n recomendaciones", rmse_grupo)
print("MAE para las top n recomendaciones", mae_grupo)


Progreso: 10000/375480 (2.66%)
Progreso: 20000/375480 (5.33%)
Progreso: 30000/375480 (7.99%)
Progreso: 40000/375480 (10.65%)
Progreso: 50000/375480 (13.32%)
Progreso: 60000/375480 (15.98%)
Progreso: 70000/375480 (18.64%)
Progreso: 80000/375480 (21.31%)
Progreso: 90000/375480 (23.97%)
RMSE para las top n recomendaciones 2.2638462845343543
MAE para las top n recomendaciones 2.25


Los códigos para random se adaptaron de un codigo inicial creado, la adaptación de este código se encuentra aquí: https://chatgpt.com/share/68e00c31-dbf8-8006-bacd-84f0296d467c


In [16]:
from sklearn.metrics import ndcg_score

# Asegúrate de que df_eval está definido como en el paso anterior.
# df_eval ya contiene las columnas: userID, itemID, rating, label, y score aleatorio.

print("\nCreando grupos sintéticos...")
# Usamos solo usuarios con al menos 10 interacciones para formar grupos más robustos
user_counts = df_eval['userID'].value_counts()
valid_users = user_counts[user_counts >= 10].index.tolist()

np.random.seed(42)
num_groups = 1000
group_size = 4
# Asegúrate de que hay suficientes usuarios válidos para crear los grupos
if len(valid_users) < group_size * num_groups:
    print(f"Advertencia: No hay suficientes usuarios únicos ({len(valid_users)}) para crear {num_groups} grupos sin reemplazo. Se crearán menos grupos.")
    num_groups = len(valid_users) // group_size

groups = [np.random.choice(valid_users, group_size, replace=False) for _ in range(num_groups)]
print(f"Se crearon {len(groups)} grupos sintéticos de tamaño {group_size}.")


print("\nAgregando predicciones para cada grupo...")
all_group_recs = []
for group_id, user_ids in enumerate(groups):
    group_predictions = df_eval[df_eval['userID'].isin(user_ids)]
    item_scores_per_group = group_predictions.groupby('itemID').agg(
        avg_score=('score', 'mean'),
        min_score=('score', 'min'),
        max_score=('score', 'max'),
        group_label=('label', lambda x: 1 if all(x == 1) else 0)
    ).reset_index()
    item_scores_per_group['group_id'] = group_id
    all_group_recs.append(item_scores_per_group)

df_group_eval = pd.concat(all_group_recs, ignore_index=True)
print("Agregación completada.")

# --- Evaluación de Estrategias con Todas las Métricas ---
strategies = {
    'Average': 'avg_score',
    'Least Misery': 'min_score',
    'Most Pleasure': 'max_score'
}

group_results = []
K_values = [10]

for strategy_name, score_column in strategies.items():
    print(f"\nEvaluando estrategia (Random): {strategy_name}...")
    df_strategy_eval = df_group_eval[['group_id', 'itemID', 'group_label']].copy()
    df_strategy_eval.rename(columns={'group_label': 'label'}, inplace=True)
    df_strategy_eval['score'] = df_group_eval[score_column]

    grouped_strategy = df_strategy_eval.groupby('group_id')

    for k in K_values:
        # Métricas existentes
        metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
        avg_precision = np.mean([m[0] for m in metrics])
        avg_recall = np.mean([m[1] for m in metrics])
        avg_ndcg = grouped_strategy.apply(lambda x: ndcg_at_k(x, k)).mean()

        # Nuevas métricas
        avg_novelty = grouped_strategy.apply(lambda x: novelty_at_k(x, k, item_popularity_prob)).mean()
        avg_diversity = grouped_strategy.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()

        group_results.append({
            'Model': 'Random',
            'Strategy': strategy_name,
            'K': k,
            'Precision@K': avg_precision,
            'Recall@K': avg_recall,
            'nDCG@K': avg_ndcg,
            'Novelty@K': avg_novelty,
            'Diversity@K': avg_diversity
        })

group_results_df = pd.DataFrame(group_results)

print("\n--- Resultados de Evaluación Grupal para Random ---")
print(group_results_df)


Creando grupos sintéticos...
Advertencia: No hay suficientes usuarios únicos (20) para crear 1000 grupos sin reemplazo. Se crearán menos grupos.
Se crearon 5 grupos sintéticos de tamaño 4.

Agregando predicciones para cada grupo...
Agregación completada.

Evaluando estrategia (Random): Average...

Evaluando estrategia (Random): Least Misery...


/tmp/ipython-input-3059351082.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3059351082.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_ndcg = grouped_strategy.apply(lambda x: ndcg_at_k(x, k)).mean()
/tmp/ipython-input-3059351082.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping


Evaluando estrategia (Random): Most Pleasure...

--- Resultados de Evaluación Grupal para Random ---
    Model       Strategy   K  Precision@K  Recall@K    nDCG@K  Novelty@K  \
0  Random        Average  10         0.42  0.188332  0.628447  13.683819   
1  Random   Least Misery  10         0.42  0.188332  0.628447  13.683819   
2  Random  Most Pleasure  10         0.42  0.188332  0.628447  13.683819   

   Diversity@K  
0     0.928452  
1     0.928452  
2     0.928452  


/tmp/ipython-input-3059351082.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_diversity = grouped_strategy.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()
/tmp/ipython-input-3059351082.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3059351082.py:62: DeprecationWarning: DataFrameGroupBy.apply 